# Requirements

tensorflow 2
petastorm==0.9.0
pyarrow==0.15.0

Node type: one driver and two workers. Databricks recommends using GPU instances.

```python
!pip install pyspark
!pip install petastorm
!pip install pyarrow
!pip install horovod[tensorflow,keras,pytorch,mxnet,spark]
```
Running models for development 283Mb shape images train-shapes-dev.tgz representing a 10% of the dataset


In [1]:
import tensorflow as tf

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from petastorm import spark
from petastorm.spark import SparkDatasetConverter, make_spark_converter
import io
import numpy as np
import tensorflow as tf
from PIL import Image
from petastorm import TransformSpec
from tensorflow import keras
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
 
from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK


In [3]:
IMG_SHAPE = (1024, 1024, 3)
BATCH_SIZE = 32
NUM_EPOCHS = 5

In [4]:
train_file = "train-shapes-dev.tgz"
keras_ds_fname = "train_shapes_dev"

Download training data set from hdfs

In [5]:
import pathlib
dataset_url = f"http://localhost:9870/webhdfs/v1/random-dot/train-data/{train_file}?op=OPEN"
data_dir = tf.keras.utils.get_file(origin=dataset_url,
                                   fname=keras_ds_fname,
                                   untar=True)
data_dir = pathlib.Path(data_dir)

297093698/297093698 [==============================] - 6s 0us/step


In [12]:
# Start spark session and read data into delta DataFrame

In [14]:
conf = SparkConf().setAppName('random-dot').setMaster('spark://16087a9ad851:7077')
spark = SparkSession.builder.getOrCreate()
df = spark.read.format("delta").load(data_dir).select(col("content"), col("label"))

NameError: name 'SparkConf' is not defined